Het samenvoegen van verschillende datasets

Het doel van dit notebook is om verschillende datasets samen te voegen zodat er één dataset ontstaat die gebruikt kan worden om analyses uit te voeren en een machine learning model mee te trainen. 

Voordat de data verzameld is, is er gekeken naar de verschillende features die invloed kunnen hebben op de energieconsumptie. De maandelijkse en wekelijkse data van de gevallen regen en temperatuur gerelateerde features hebben veel invloed (Guo, 2018). Daarnaast zijn er ook onderzoeken uitgevoerd waarbij de gemiddelde temperatuur, minimum en maximum temperatuur, vochtigheid, zonnestraling en windsnelheid meegenomen zijn (Zhao, 2012). Daarom zijn de factoren wind, gemiddelde temperatuur en het aantal zonuren per dag meegenomen in het trainen van het machine learning model. Deze features worden daarom ook in deze notebook opgehaald. De weer data is afkomstig van het KNMI, locatie de Bilt. Locatie de Bilt is gekozen omdat deze locatie als graadmeter in Nederland kan worden gezien.

(Ook zijn er andere features opgehaald in het bestand "ML-Regression_twzi_QSP.ipynb" wordt uitgelegd waarom deze features meegenomen zijn)

In [1]:
import pandas as pd
import datetime

Als eerste worden de datasets van de verschillende features opgehaald. 

In [11]:
temperatuur = pd.read_csv('Dataset_temperatuur.csv')
wind = pd.read_csv('WeatherMontlyAverage.csv')
zonuren = pd.read_csv('zonuren1.csv')
energie = pd.read_csv('MontlyEnergy.csv')
inflatie = pd.read_csv('Inflatie.csv', delimiter=";")
straling = pd.read_csv('zonuren_straling.csv', sep = ',')
percentage_zonuren = pd.read_csv('zonuren_percentage.csv', sep = ',')

Vervolgens wordt in iedere feature de kolom naam veranderd naar Datum. Dit wordt gedaan zodat de datasets op de Datum kolom worden samengevoegd en dit kan alleen wanneer de datum in hetzelfde format staat en de kolom naam hetzelfde is. Daarnaast worden de overige kolommen uit de dataset verwijderd, zodat de samengevoegde dataset overzichtelijk is.

In [12]:
temperatuur.rename(columns = {'DateTime':'Datum'}, inplace = True)
temperatuur.drop(columns=['Unnamed: 0','TN', 'TX'], inplace=True)
print(temperatuur.head())

     Datum         TG
0  2000-01   4.329032
1  2000-02   5.855172
2  2000-03   6.796774
3  2000-04   9.986667
4  2000-05  14.732258


In [13]:
wind.rename(columns = {'YYYYMMDD':'Datum'}, inplace = True)
print(wind.head())

     Datum     FHVEC
0  2000-01  3.980645
1  2000-02  4.296552
2  2000-03  3.677419
3  2000-04  2.840000
4  2000-05  2.900000


In [14]:
energie['Perioden'] = energie['Perioden'].apply(lambda x: datetime.datetime.strptime(x, "%YMM%m").strftime("%Y-%m"))
energie.rename(columns = {'Perioden':'Datum'}, inplace = True)
energie.drop(columns=['Unnamed: 0','ID'], inplace=True)
print(energie.head())

     Datum  NettoVerbruikBerekend_30
0  2000-01                      8847
1  2000-02                      8235
2  2000-03                      8738
3  2000-04                      7912
4  2000-05                      8221


In [15]:
inflatie.rename(columns = {'Perioden':'Datum'}, inplace = True)
inflatie.drop(columns=['Jaar','Maand','CPI_energie-100', 'CPI(afgeleid)_energie ', 'CPI_algemeen', 
                       'CPI(algemeen)_afgeleid', 'CPI_elektriciteit', 'CPI(elektriciteit)_afgeleid', 
                       'CPI_gas', 'CPI(gas)_afgeleid', 'CPI_energiejaarlijks'], inplace=True)
print(inflatie.head())

     Datum  CPI_energie
0  2000-01        52.22
1  2000-02        52.29
2  2000-03        52.73
3  2000-04        52.74
4  2000-05        52.74


In [16]:
straling.rename(columns = {'    Q':'Q'}, inplace = True)
straling.drop(columns=['zonuren_dag'], inplace=True)
print(straling.head())

     Datum            Q
0  2000-01   201.774194
1  2000-02   484.310345
2  2000-03   629.258065
3  2000-04  1263.600000
4  2000-05  1701.903226


In [17]:
percentage_zonuren.rename(columns = {'    SP':'SP'}, inplace = True)
percentage_zonuren.drop(columns=['zonuren_dag'], inplace=True)
print(percentage_zonuren.head())

     Datum         SP
0  2000-01  21.838710
1  2000-02  36.827586
2  2000-03  21.967742
3  2000-04  37.066667
4  2000-05  43.354839


Met de functie merge worden de datasets van alle features samengevoegd. 

In [18]:
alle_datasets = temperatuur.merge(wind, on='Datum') \
                                    .merge(zonuren, on='Datum')\
                                    .merge(energie, on='Datum')\
                                    .merge(inflatie, on='Datum')\
                                    .merge(straling, on='Datum')\
                                    .merge(percentage_zonuren, on='Datum')
alle_datasets

,Datum,TG,FHVEC,zonuren_dag,NettoVerbruikBerekend_30,CPI_energie,Q,SP
0,2000-01,4.329032,3.980645,1.806452,8847,52.22,201.774194,21.838710
1,2000-02,5.855172,4.296552,3.682759,8235,52.29,484.310345,36.827586
2,2000-03,6.796774,3.677419,2.583871,8738,52.73,629.258065,21.967742
3,2000-04,9.986667,2.840000,5.116667,7912,52.74,1263.600000,37.066667
4,2000-05,14.732258,2.900000,6.780645,8221,52.74,1701.903226,43.354839
...,...,...,...,...,...,...,...,...
259,2021-08,16.935484,2.677419,5.325806,8981,117.17,1391.612903,36.258065
260,2021-09,15.930000,2.160000,5.940000,9030,123.29,1165.900000,46.433333
261,2021-10,11.612903,3.045161,3.945161,9410,136.56,619.709677,36.774194
262,2021-11,7.380000,2.383333,1.946667,9678,163.61,276.566667,22.033333


De samengevoegde dataset wordt opgeslagen.

In [19]:
alle_datasets.to_csv('alle_datasets.csv')

Bronnenlijst
Guo, Z. (2018, juli 17). A deep learning model for short-term power load and probability density forecasting. Opgehaald van ScienceDirect: https://www.sciencedirect.com/science/article/pii/S0360544218313872
Zhao, H.-x. (2012, april 27). A review on the prediction of building energy consumption. Opgehaald van ScienceDirect: https://www.sciencedirect.com/science/article/pii/S1364032112001438